<a href="https://colab.research.google.com/github/metalnom/Colab_01/blob/master/note02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝 구현
- 은닉층 활성화 함수: ReLU
- 출력층 활성화 함수: Softmax
- 손실 함수: 교차 엔트로피 오차
- 최적화 알고리즘: 확률적 경사 하강법
- 배치 사이즈: 8
- 은닉층의 뉴런 수: 25
- input_data는 평균 0, 표준편차 1로 표준화
- correct_data는 원핫인코딩